### onedrive내 train data를 가공해 손글씨 raw파일 추출을 위한 notebook
  - input: 다양한 size의 image, x, y, w, h정보 label
  - output: 손글씨 형태만 crop된 이미지(파일명을 텍스트로 저장)

### onedrive

In [ ]:
!wget https://downloads.rclone.org/v1.57.0/rclone-v1.57.0-linux-amd64.deb
!apt install ./rclone-v1.57.0-linux-amd64.deb

--2022-10-25 00:50:00--  https://downloads.rclone.org/v1.57.0/rclone-v1.57.0-linux-amd64.deb
Resolving downloads.rclone.org (downloads.rclone.org)... 95.217.6.16, 2a01:4f9:c012:7154::1
Connecting to downloads.rclone.org (downloads.rclone.org)|95.217.6.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14610238 (14M) [application/vnd.debian.binary-package]
Saving to: ‘rclone-v1.57.0-linux-amd64.deb’

rclone-v1.57.0-linu 100%[===================>]  13.93M  4.62MB/s    in 3.0s    

2022-10-25 00:50:04 (4.62 MB/s) - ‘rclone-v1.57.0-linux-amd64.deb’ saved [14610238/14610238]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'rclone' instead of './rclone-v1.57.0-linux-amd64.deb'
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  rclone
0 upgraded, 1 newly installed, 0 to rem

In [ ]:
!rclone config

In [ ]:
!mkdir onedrive
!nohup rclone --vfs-cache-mode writes mount onedrive: ./onedrive &    

nohup: appending output to 'nohup.out'


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### imwrite

In [ ]:
# 한글 저장을 위한 함수
def imwrite(filename, img, params=None):
    try:
        ext = os.path.splitext(filename)[1]
        result, n = cv2.imencode(ext, img, params)

        if result:
            with open(filename, mode='w+b') as f:
                n.tofile(f)
            return True
        else:
            return False
    except Exception as e:
        print(e)
        return False

### financial ocr

In [ ]:
import zipfile
import os

os.makedirs('images', exist_ok=True)
os.makedirs('images/insure', exist_ok=True)

source_file = r'/content/onedrive/playdata/final_project/train/TS2.zip'
ann_file = r'/content/onedrive/playdata/final_project/train/TL2.zip'

with zipfile.ZipFile(source_file, 'r') as zf:
	zf.extractall('images/insure')

with zipfile.ZipFile(ann_file, 'r') as zf:
	zf.extractall('images/insure')

In [ ]:
import zipfile
import os

os.makedirs('images', exist_ok=True)
os.makedirs('images/insure', exist_ok=True)

source_file = r'/content/onedrive/playdata/final_project/train/TS2.zip'
ann_file = r'/content/onedrive/playdata/final_project/train/TL2.zip'

with zipfile.ZipFile(source_file, 'r') as zf:
	zf.extractall('images/insure')

with zipfile.ZipFile(ann_file, 'r') as zf:
	zf.extractall('images/insure')

In [ ]:
import cv2
import numpy as np
import os
import shutil
import json
import matplotlib.pyplot as plt

In [ ]:
for j in os.listdir(r'/content/images/insure/result/insure/annotations'):
  with open(f'/content/images/insure/result/insure/annotations/{j}', encoding='UTF8') as f:
    json_object = json.load(f)
  img = cv2.imread(f'/content/images/insure/result/insure/images/{json_object["name"]}')
  obj = json_object['annotations'][0]
  for polygon, bbox in zip(obj['polygons'], obj['bbox']):
    if polygon['type'] != 2:
        continue
    x1, y1 = int(bbox['x']), int(bbox['y'])
    x2, y2 = x1 + int(bbox['width']), y1 + int(bbox['height'])
#     print(polygon, bbox)
#     clone = cv2.rectangle(clone, (x1, y1), (x2, y2), (0,0,255), 10)
    cropped_image = img[y1:y2+1, x1:x2+1]
    # print(polygon["text"])
    imwrite(f'/content/drive/MyDrive/playdata/textcrop/{polygon["text"]}-{polygon["id"]}.jpg', cropped_image)

FileNotFoundError: ignored

In [ ]:
shutil.copytree('/content/onedrive/playdata/final_project/validation/insure', 'insure')

'insure'

In [ ]:
for j in os.listdir(r'/content/insure/annotations'):
  with open(f'/content/insure/annotations/{j}', encoding='UTF8') as f:
    json_object = json.load(f)
  img = cv2.imread(f'/content/insure/images/{json_object["name"]}')
  obj = json_object['annotations'][0]
  for polygon, bbox in zip(obj['polygons'], obj['bbox']):
    if polygon['type'] != 2:
        continue
    x1, y1 = int(bbox['x']), int(bbox['y'])
    x2, y2 = x1 + int(bbox['width']), y1 + int(bbox['height'])
#     print(polygon, bbox)
#     clone = cv2.rectangle(clone, (x1, y1), (x2, y2), (0,0,255), 10)
    cropped_image = img[y1:y2+1, x1:x2+1]
    # print(polygon["text"])
    imwrite(f'/content/drive/MyDrive/playdata/val_textcrop/{polygon["text"]}-{polygon["id"]}.jpg', cropped_image)

In [ ]:
shutil.rmtree('/content/images')

### handwriting ocr

In [ ]:
import zipfile
import os
import cv2
import numpy as np
import shutil
import json
import matplotlib.pyplot as plt

In [ ]:
ziplist = [#r'/content/onedrive/playdata/final_project/handwriting/train/handwriting_ocr_ts6.zip',
           r'/content/onedrive/playdata/final_project/handwriting/train/handwriting_ocr_ts7.zip',
           r'/content/onedrive/playdata/final_project/handwriting/train/handwriting_ocr_ts8.zip']

annfile = r'/content/onedrive/playdata/final_project/handwriting/train/handwriting_ocr_tl.zip'

In [ ]:
if not os.path.isdir('ann'):
	os.makedirs('ann', exist_ok=True)
	with zipfile.ZipFile(annfile, 'r') as zf:
		zf.extractall('ann')
 
for i, zip in enumerate(ziplist):
	os.makedirs('images', exist_ok=True)
	if not os.path.isdir(f'crop{i}'):
		os.makedirs(f'crop{i}', exist_ok=True)
		with zipfile.ZipFile(zip, 'r') as zf:
			zf.extractall('images')
	for j in os.listdir(r'/content/images/HW-OCR/4.Validation/P.Paper/R.Free'):
		with open(f'/content/ann/label/HW-OCR/4.Validation/P.Paper/R.Free/{os.path.splitext(j)[0]}.json', encoding='UTF8') as f:
			json_object = json.load(f)
		img = cv2.imread(f'/content/images/HW-OCR/4.Validation/P.Paper/R.Free/{j}', cv2.IMREAD_GRAYSCALE)
		obj = json_object['bbox']
		for bbox in obj:
			x1, y1 = int(min(bbox['x'])), int(min(bbox['y']))
			x2, y2 = int(max(bbox['x'])), int(max(bbox['y']))
			cropped_image = img[y1:y2+1, x1:x2+1]
			print(f'/content/crop{i}/{json_object["Images"]["identifier"]}-{bbox["data"]}.jpg')
			imwrite(f'/content/crop{i}/{json_object["Images"]["identifier"]}-{bbox["data"]}.jpg', cropped_image)
	shutil.rmtree('images')
	with zipfile.ZipFile(f'/content/drive/MyDrive/final_project/crop{i}.zip', 'w') as zf:  # "w": write 모드
		for file in os.listdir(f'crop{i}'):
					zf.write(os.path.join(f'crop{i}', file), compress_type=zipfile.ZIP_DEFLATED)
	shutil.rmtree(f'crop{i}')